### Load Libraries

In [1]:
import os 
import pandas as pd
import numpy as np

### Connecting Google Colab to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path_data = '/content/drive/MyDrive/Data/'

Mounted at /content/drive


Now, colab can access the datasets from Google Drive

### Create Monthly Sales Transaction Report

In [3]:
order = pd.read_csv(path_data + 'olist_orders_dataset.csv')

order_date = ['order_purchase_timestamp',
            'order_approved_at',
            'order_delivered_carrier_date',
            'order_delivered_customer_date',
            'order_estimated_delivery_date']

for col in order_date:
  order[col] = pd.to_datetime(order[col])

order['Tahun Transaksi'] = order['order_purchase_timestamp'].dt.year
order['Bulan Transaksi'] = order['order_purchase_timestamp'].dt.month_name()

order_items = pd.read_csv(path_data + 'olist_order_items_dataset.csv')
order = order[order['order_id'].isin(order_items['order_id'].unique())]

order_price = order_items.groupby(['order_id']).agg({'price':'sum'}).reset_index()
order = order.merge(order_price)

order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,Tahun Transaksi,Bulan Transaksi,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017,October,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,2018,July,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,2018,August,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,2017,November,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2018,February,19.90


In [4]:
report_order = order.groupby(['Tahun Transaksi','Bulan Transaksi']).agg({'order_id':'count', 'price':'sum'}).unstack()
report_order

order_id                                                    \
Bulan Transaksi    April  August December February January    July    June   
Tahun Transaksi                                                              
2016                 NaN     NaN      1.0      NaN     NaN     NaN     NaN   
2017              2391.0  4293.0   5624.0   1733.0   789.0  3969.0  3217.0   
2018              6934.0  6452.0      NaN   6694.0  7220.0  6273.0  6160.0   

                                          ...      price             \
Bulan Transaksi   March     May November  ...   December   February   
Tahun Transaksi                           ...                         
2016                NaN     NaN      NaN  ...      10.90        NaN   
2017             2641.0  3660.0   7451.0  ...  743914.17  247303.02   
2018             7188.0  6853.0      NaN  ...        NaN  844178.71   

                                                                        \
Bulan Transaksi    January       July       June      March        May   
Tahun Transaksi                                                          
2016                   NaN        NaN        NaN        NaN        NaN   
2017             120312.87  498031.48  433038.60  374344.30  506071.14   
2018             950030.36  895507.22  865124.31  983213.44  996517.68   

                                                   
Bulan Transaksi    November    October  September  
Tahun Transaksi                                    
2016                    NaN   49507.66     267.36  
2017             1010271.37  664219.43  624401.69  
2018                    NaN        NaN     145.00  

[3 rows x 24 columns]

### Create Monthly Summary Reviews

In [5]:
review = pd.read_csv(path_data + 'olist_order_reviews_dataset.csv')

review_date = ['review_creation_date',
            'review_answer_timestamp']

for col in review_date:
  review[col] = pd.to_datetime(review[col])

review['Tahun Review'] = review['review_creation_date'].dt.year
review['Bulan Review'] = review['review_creation_date'].dt.month_name()

review = review[review['order_id'].isin(order_items['order_id'].unique())]
review.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,Tahun Review,Bulan Review
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,2018,January
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,2018,March
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,2018,February
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,2017,April
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,2018,March


In [6]:
report_review = review.groupby(['review_score','Tahun Review','Bulan Review']).agg({'order_id':'count'}).unstack()
report_review

order_id                                            \
Bulan Review                 April  August December February January    July   
review_score Tahun Review                                                      
1            2016              NaN     NaN     28.0      NaN     NaN     NaN   
             2017            232.0   361.0   1167.0    100.0    17.0   298.0   
             2018           1146.0   821.0      NaN    785.0   708.0   466.0   
2            2016              NaN     NaN      NaN      NaN     NaN     NaN   
             2017             71.0   140.0    287.0     39.0     3.0   111.0   
             2018            277.0   245.0      NaN    203.0   223.0   128.0   
3            2016              NaN     NaN      1.0      NaN     NaN     NaN   
             2017            194.0   349.0    675.0    122.0    23.0   301.0   
             2018            614.0   638.0      NaN    570.0   555.0   399.0   
4            2016              NaN     NaN      1.0      NaN     NaN     NaN   
             2017            399.0   848.0   1561.0    275.0    35.0   683.0   
             2018           1374.0  1697.0      NaN   1157.0  1204.0  1018.0   
5            2016              NaN     NaN     14.0      NaN     NaN     NaN   
             2017           1143.0  2788.0   4278.0    886.0   160.0  2099.0   
             2018           3925.0  5554.0      NaN   3400.0  3508.0  3641.0   

                                                                              
Bulan Review                 June   March     May November October September  
review_score Tahun Review                                                     
1            2016             NaN     NaN     NaN     25.0    24.0       NaN  
             2017           340.0   279.0   385.0    467.0   396.0     388.0  
             2018           620.0  1536.0   714.0      NaN     NaN       NaN  
2            2016             NaN     NaN     NaN      3.0     4.0       NaN  
             2017           107.0    73.0   131.0    134.0   128.0     127.0  
             2018           209.0   316.0   212.0      NaN     NaN       NaN  
3            2016             NaN     NaN     NaN      8.0    14.0       NaN  
             2017           311.0   218.0   329.0    439.0   339.0     314.0  
             2018           524.0   725.0   572.0      NaN     NaN       NaN  
4            2016             NaN     NaN     NaN     19.0    31.0       NaN  
             2017           682.0   480.0   764.0    936.0   927.0     859.0  
             2018          1269.0  1459.0  1488.0      NaN     NaN       NaN  
5            2016             NaN     NaN     NaN     36.0   106.0       NaN  
             2017          2000.0  1388.0  2107.0   2771.0  2624.0    2512.0  
             2018          4109.0  3786.0  4513.0      NaN     NaN       NaN

### Create Monthly Top 5 Products

In [7]:
product = pd.read_csv(path_data + 'X.csv')
order_items = order_items[order_items['product_id'].isin(product['product_id'].unique())]
product = product[['product_id','product_category_name']].reset_index()
order_items = order_items.merge(product).reset_index()
order_items = order_items.merge(order)
order_items = order_items[['order_id','order_item_id','product_id','product_category_name','Tahun Transaksi','Bulan Transaksi']]
order_items.groupby(['Tahun Transaksi','Bulan Transaksi','product_category_name']).agg({'product_category_name':'count'}).unstack()

order_items.head()

,order_id,order_item_id,product_id,product_category_name,Tahun Transaksi,Bulan Transaksi
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,2017,September
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,2017,June
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,2018,May
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,2017,August
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,2017,August


In [8]:
report_product = order_items.groupby(['Tahun Transaksi', 'Bulan Transaksi','product_category_name']).agg({'product_category_name':'count'})
report_product

product_category_name
Tahun Transaksi Bulan Transaksi product_category_name                                                
2016            December        fashion_bolsas_e_acessorios                                         1
                October         alimentos                                                           1
                                audio                                                               2
                                automotivo                                                         10
                                bebes                                                               8
...                                                                                               ...
2018            May             tablets_impressao_imagem                                            1
                                telefonia                                                         228
                                telefonia_fixa                                                     10
                                utilidades_domesticas                                             414
                September       moveis_cozinha_area_de_servico_jantar_e_jardim                      1

[1235 rows x 1 columns]

### Exporting Report to Google Drive

In [9]:
with pd.ExcelWriter(path_data + 'Sales Transaction Report.xlsx') as writer:    
    # Write each dataframe to a different worksheet.
    report_order.to_excel(writer, sheet_name='Order')
    report_review.to_excel(writer, sheet_name='Review')
    report_product.to_excel(writer, sheet_name='Product')